In [3]:
# Import pacakge
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

# 국내주식갤
# gal_id = "krstock"
# page_no = 4300 # << 4300이 최대페이지 / 2025년 5월 24일까지 가능

# 코스피갤
gal_id = "kospi"
page_no = 11000 # 2024년 12월 11일 (넉넉)

# 주식갤
# gal_id = "neostock"
# page_no = 10000 # 2024년 12월 17일 (넉넉)

# 삼성전자갤 - 근데 주식 아닌 데 까지 있음
# gal_id = "samsungelectron"
# page_no = 7 # << 7이 최대페이지 / 2023년 6월 23일까지

url = f"https://gall.dcinside.com/mgallery/board/lists/?id={gal_id}&page={page_no}"

# 접속 정보를 입력
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/143.0.0.0 Safari/537.36"
}

response = requests.get(url, headers=HEADERS)
soup = bs(response.text, "lxml")

table = soup.select_one("table.gall_list")

columns = [th.text.strip() for th in table.select("th")]
rows = []
for tr in table.select("tbody tr"):
    tds = tr.select("td")

    row = [td.text.strip() for td in tds]
    if row[1] == "일반" : # 일반 게시물만 크롤링
        rows.append(row)
    else :
        continue
    
# DataFrame 생성
stock_table = pd.DataFrame(rows, columns=columns)

print(len(stock_table))
stock_table.head()

50


,번호,말머리,제목,글쓴이,작성일,조회,추천
0,4325356,일반,양자컴 나오면 즉시 뒤지는 애들,찢황만세(1.235),24.12.11,95,3
1,4325355,일반,찢 깜빵가냐?\n[2],ㅇㅇ(118.235),24.12.11,66,0
2,4325354,일반,형두창단조부활ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ,HNH,24.12.11,45,0
3,4325353,일반,HBM 퀼통과해도 삼전 주가 구데기일듯\n[2],ㅇㅇ(223.38),24.12.11,88,0
4,4325352,일반,신성 평단 105000 오늘 다 정리했다,찢황만세(211.235),24.12.11,277,0


In [23]:
# Import pacakge
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import time

# 코스피갤
gal_id = "kospi"
# page_no = 11000 # 2024년 12월 11일 (넉넉)

# 접속 정보
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/143.0.0.0 Safari/537.36"
}

# 조회할 페이지
start_page = 1
end_page = 10

posts = {
    "post_id" : [], # 게시글 id
    "subject" : [],
    "title" : [], # 제목
    "date" : [], # 작성일
    "view_count" : [], # 조회수
    "recommend_count" : [], # 추천수
    "content" : [] # 게시글 내용
}

for page_no in range(start_page, end_page+1, 1) :
    url = f"https://gall.dcinside.com/mgallery/board/lists/?id={gal_id}&page={page_no}"
    
    response = requests.get(url, headers=HEADERS)
    soup = bs(response.text, "lxml")

    table = soup.select_one("table.gall_list")

    for tr in table.select("tbody tr"): # tr : 게시글 목록에서 한 row
        # tds = tr.select("td") # td(s) : 번호 / 말머리 / 제목 / 글쓴이 / 작성일 / 조회 / 추천
        if tr.select_one("td.gall_subject").text.strip() in ("설문", "AD", "공지") :
            continue
        # if tr.select_one("td.gall_date"). not in () # 날짜가 원하는 기간이 아닐 때 continue 또는 아예 크롤링 중단하는 코드 추가해야함
        
        posts["subject"].append(tr.select_one("td.gall_subject").text.strip())
        
        posts["post_id"].append(tr.select_one("td.gall_num").text.strip())
        posts["title"].append(tr.select_one("td.gall_tit a").text.strip())
        posts["date"].append(tr.select_one("td.gall_date").text.strip())
        posts["view_count"].append(tr.select_one("td.gall_count").text.strip())
        posts["recommend_count"].append(tr.select_one("td.gall_recommend").text.strip())
        
        # 제목에 달린 링크로 게시글 가져오기
        post_link = tr.select_one("td.gall_tit a").attrs["href"]
        post_url = f"https://gall.dcinside.com{post_link}"
        post_response = requests.get(post_url, headers=HEADERS)
        post_soup = bs(post_response.text, "lxml")
        
        # print(post_soup.select_one("div.write_div"))
        
        posts["content"].append(post_soup.select_one("div.write_div").text.strip())
        
        
df = pd.DataFrame(posts)

print(len(df))
df.head()

df.to_csv("./test_data.csv")

495
